In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mytools
import pandas as pd
import geopandas as gpd

# ---导入数据---

In [3]:
path = 'G:/1-规划/1其他工作输出/20200221问题点输出解决方案/2-处理过程使用数据/'

In [4]:
wenti = pd.read_csv('{}wenti-对比使用.csv'.format(path),encoding='gbk')
weikaitong = pd.read_csv('{}规划未开通站点.csv'.format(path),encoding='gbk')
xianwang =pd.read_csv('{}现网小区.csv'.format(path),encoding='gbk')

# ---数据预处理---

In [5]:
wenti_cahngjing = mytools.gisn.points_coverage_merge(wenti,'lon','lat',
                                                        coverage='G:/1-规划/图层/全省区域规划图层20180510/全省区域规划图层20180510/全省区域规划图层20180510.TAB',
                                                        df_merge_coverage_columns=['区域类型'],
                                                        merge_only_right_mark='农村')

In [6]:
wenti_cahngjing.loc[wenti_cahngjing['区域类型']=='主城区','优化半径']=100
wenti_cahngjing.loc[wenti_cahngjing['区域类型']=='一般城区','优化半径']=127
wenti_cahngjing.loc[wenti_cahngjing['区域类型']=='县城','优化半径']=121
wenti_cahngjing.loc[wenti_cahngjing['区域类型']=='乡镇','优化半径']=289
wenti_cahngjing.loc[wenti_cahngjing['区域类型']=='农村','优化半径']=500

In [7]:
wenti_yh_buff = mytools.gisn.add_buffer_df(wenti_cahngjing,'lon','lat',buff_col='优化半径')

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [8]:
xianwang_p = mytools.gisn.add_points(xianwang,'RRU经度','RRU纬度')

In [9]:
weikaitong_cahngjing = mytools.gisn.points_coverage_merge(weikaitong,'规划经度','规划纬度',
                                                        coverage='G:/1-规划/图层/全省区域规划图层20180510/全省区域规划图层20180510/全省区域规划图层20180510.TAB',
                                                        df_merge_coverage_columns=['区域类型'],
                                                        merge_only_right_mark='农村')

In [10]:
weikaitong_cahngjing1 = mytools.gisn.changjing_distance(weikaitong_cahngjing,'区域类型', '系统制式')

In [11]:
weikaitong_cahngjing_buff = mytools.gisn.add_buffer_df(weikaitong_cahngjing1,'规划经度','规划纬度',buff_col='distance')

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [12]:
wenti_cahngjing_p = mytools.gisn.add_points(wenti_cahngjing,'lon','lat')

# ---生成优化方案---

In [13]:
youhua_fangan_v1 = gpd.sjoin(wenti_yh_buff,xianwang_p)

In [14]:
youhua_fangan_v1.head(1)

,id,city,qx,lon,lat,六大场景,区域类型,优化半径,geometry,index_right,小区CGI(*),小区中文名,RRU经度,RRU纬度,工作频段
0,20200324-SWMR-ES-000101,恩施,建始县,109.735335,30.630336,居民区,农村,500.0,"POLYGON ((109.74051 30.63063, 109.74052 30.630...",166707,460-00-582473-130,建始高坪国际教育-HLH-2,109.733711,30.629471,F


In [15]:
youhua_fangan_v12 = mytools.gisn.distancea_df(youhua_fangan_v1,'lon','lat','RRU经度','RRU纬度',columns_name='对应距离_优化')

In [16]:
youhua_fangan_v12.columns

Index(['id', 'city', 'qx', 'lon', 'lat', '六大场景', '区域类型', '优化半径', 'geometry',
       'index_right', '小区CGI(*)', '小区中文名', 'RRU经度', 'RRU纬度', '工作频段',
       '对应距离_优化'],
      dtype='object')

In [17]:
youhua_fangan_v13 = youhua_fangan_v12.sort_values('对应距离_优化')

In [18]:
youhua_fangan_v13.loc[youhua_fangan_v13['工作频段']=='FDD900','zhishi_1']=1

In [19]:
youhua_fangan_v13.loc[youhua_fangan_v13['工作频段']=='FDD1800','zhishi_1']=2

In [20]:
youhua_fangan_v14 = youhua_fangan_v13.sort_values('zhishi_1')

In [21]:
youhua_fangan_v15 = youhua_fangan_v14.drop_duplicates('id')

In [22]:
youhua_fangan_v15.to_csv('{}优化方案-对比.csv'.format(path),encoding='gbk',index=False)

# ---生成建设方案---

In [23]:
jianshe_fangan_v1 = gpd.sjoin(wenti_cahngjing_p,weikaitong_cahngjing_buff)

In [24]:
jianshe_fangan_v2 = mytools.gisn.distancea_df(jianshe_fangan_v1,'lon','lat','规划经度','规划纬度',columns_name='相对距离')

In [25]:
jianshe_fangan_v3 = jianshe_fangan_v2.sort_values('相对距离').drop_duplicates('id')

In [26]:
jianshe_fangan_v3.to_csv('{}建设方案-对比.csv'.format(path),encoding='gbk',index=False)

# ---生成规划方案---

In [27]:
you_jian = jianshe_fangan_v3[['id','city']].append(youhua_fangan_v15[['id','city']])

In [28]:
aaa = wenti.merge(you_jian,how='outer',on='id',indicator=True)

In [29]:
aaa.groupby('_merge').count()

,id,city_x,qx,lon,lat,六大场景,city_y
_merge,,,,,,,
left_only,529,529,524,529,529,529,0
right_only,0,0,0,0,0,0,0
both,6117,6117,6101,6117,6117,6117,6117
